In [1]:
import os
import zipfile
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
def normalize(volume):
    #Adjust to normalize the scans to be between 0 and 1
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume


In [ ]:
# Read and process the scans.
# Each scan is resized across height, width, and depth and rescaled.
bad_pairs = np.array([process_scan(path) for path in abnormal_scan_paths])
good_pairs = np.array([process_scan(path) for path in normal_scan_paths])

# For the good pairs
# assign 1, for the bad ones assign 0.
bad_labels = np.array([0 for _ in range(len(bad_pairs))])
good_labels = np.array([1 for _ in range(len(good_pairs))])

# Split data in the ratio 70-30 for training and validation.
x_train = np.concatenate((bad_pairs[:70], good_pairs[:70]), axis=0)
y_train = np.concatenate((bad_labels[:70], good_labels[:70]), axis=0)
x_val = np.concatenate((bad_pairs[70:], good_pairs[70:]), axis=0)
y_val = np.concatenate((bad_labels[70:], good_labels[70:]), axis=0)
print(
    "Number of samples in train and validation are %d and %d."
    % (x_train.shape[0], x_val.shape[0])
)


In [ ]:
def train_preprocessing(volume, label):
    #"""Process training data by adding a channel."""
    volume = tf.expand_dims(volume, axis=3)
    return volume, label


def validation_preprocessing(volume, label):
    #"""Process validation data by adding a channel."""
    volume = tf.expand_dims(volume, axis=3)
    return volume, label

# Define data loaders.
train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
validation_loader = tf.data.Dataset.from_tensor_slices((x_val, y_val))

batch_size = 2
# Augment the on the fly during training.
train_dataset = (
    train_loader.shuffle(len(x_train))
    .map(train_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)
# Only rescale.
validation_dataset = (
    validation_loader.shuffle(len(x_val))
    .map(validation_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)


In [1]:
def get_model(width=128, height=128, depth=64):
    #"""Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model


# Build model.
model = get_model(width=128, height=128, depth=64)
model.summary()


NameError: name 'keras' is not defined